## Importing The Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras import optimizers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


## Specifying the Working Directories

In [2]:
# Defining the path to the dataset 
base_dir = 'E:\Docs\courseRelated\AdvTopicsInML\FinalProject'

# Directories for training and validation dataset
train_dir = os.path.join(base_dir, 'dataset\\train')
val_dir = os.path.join(base_dir, 'dataset\\validation')

# Directory with training objects
train_boat_dir = os.path.join(train_dir, 'boat')
train_building_dir = os.path.join(train_dir, 'building')
train_cars_dir = os.path.join(train_dir, 'cars')
train_drones_dir = os.path.join(train_dir, 'drones')
train_group_dir = os.path.join(train_dir, 'group')
train_horseride_dir = os.path.join(train_dir, 'horseride')
train_paraglider_dir = os.path.join(train_dir, 'paraglider')
train_person_dir = os.path.join(train_dir, 'person')
train_riding_dir = os.path.join(train_dir, 'riding')
train_trucks_dir = os.path.join(train_dir, 'trucks')
train_wakeboard_dir = os.path.join(train_dir, 'wakebard')
train_whales_dir = os.path.join(train_dir, 'whales')

# Directory with validation objects
val_boat_dir = os.path.join(val_dir, 'boat')
val_building_dir = os.path.join(val_dir, 'building')
val_cars_dir = os.path.join(val_dir, 'cars')
val_drones_dir = os.path.join(val_dir, 'drones')
val_group_dir = os.path.join(val_dir, 'group')
val_horseride_dir = os.path.join(val_dir, 'horseride')
val_paraglider_dir = os.path.join(val_dir, 'paraglider')
val_person_dir = os.path.join(val_dir, 'person')
val_riding_dir = os.path.join(val_dir, 'riding')
val_trucks_dir = os.path.join(val_dir, 'trucks')
val_wakeboard_dir = os.path.join(val_dir, 'wakebard')
val_whales_dir = os.path.join(val_dir, 'whales')

## Setting Parameters for the Data Generators

In [3]:
# Specifying the number of samples in a batch
batch_size = 32

# Defining imput image size
IMG_HEIGHT = 640
IMG_WIDTH  = 360

## Preprocessing Training and Validation Datasets 

In [4]:
# Augmenting training dataset
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_data_gen = train_datagen.flow_from_directory(batch_size=batch_size,
                                                   directory=train_dir,
                                                   target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                   class_mode='sparse')

# Preprocessing validation dataset
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_data_gen = valid_datagen.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='sparse')

Found 65976 images belonging to 12 classes.
Found 27544 images belonging to 12 classes.


## Defining a Pre-Trained ResNet50 Model

In [5]:
# Loading pre-trained model from keras
from tensorflow.keras.applications import ResNet50

# Forming the convolutional base
conv_base = ResNet50(weights='imagenet',
                     include_top=False,
                     input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# Freezing the convolutional base
conv_base.trainable = False

# Stacking the dense layers
model = tf.keras.models.Sequential()
model.add(conv_base)   # Adding the conv_base as one of the layers first
model.add(tf.keras.layers.Flatten()) 
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 20, 12, 2048)      23587712  
_________________________________________________________________
flatten (Flatten)            (None, 491520)            0         
_________________________________________________________________
dense (Dense)                (None, 256)               125829376 
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 149,417,345
Trainable params: 125,829,633
Non-trainable params: 23,587,712
_________________________________________________________________


## Compiling The Model

In [6]:
%%time

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=optimizers.RMSprop(learning_rate=2e-5), 
              metrics=['accuracy'])
history_resnet50_tfkerastl = model.fit(train_data_gen, 
                                       epochs=20, 
                                       validation_data=val_data_gen)

Epoch 1/20
2062/2062 [==============================] - 2533s 1s/step - loss: nan - accuracy: 0.0547 - val_loss: nan - val_accuracy: 0.0568
Epoch 2/20
2062/2062 [==============================] - 2199s 1s/step - loss: nan - accuracy: 0.0554 - val_loss: nan - val_accuracy: 0.0568
Epoch 3/20
2062/2062 [==============================] - 2196s 1s/step - loss: nan - accuracy: 0.0559 - val_loss: nan - val_accuracy: 0.0568
Epoch 4/20
2062/2062 [==============================] - 2205s 1s/step - loss: nan - accuracy: 0.0560 - val_loss: nan - val_accuracy: 0.0568
Epoch 5/20
2062/2062 [==============================] - 2197s 1s/step - loss: nan - accuracy: 0.0546 - val_loss: nan - val_accuracy: 0.0568
Epoch 6/20
2062/2062 [==============================] - 2207s 1s/step - loss: nan - accuracy: 0.0552 - val_loss: nan - val_accuracy: 0.0568
Epoch 7/20
2062/2062 [==============================] - 2196s 1s/step - loss: nan - accuracy: 0.0563 - val_loss: nan - val_accuracy: 0.0568
Epoch 8/20
2062/2062

## Saving Loss and Accuracy Scores

In [7]:
history_resnet50_tfkerastl_df = pd.DataFrame(data=history_resnet50_tfkerastl.history)
history_resnet50_tfkerastl_df.to_csv('history_ResNet50_TFKerasTL_Python.csv', index=False)
history_resnet50_tfkerastl_df.to_csv('history_ResNet50_TFKerasTL_MATLAB.csv', header=False, index=False)